<a href="https://colab.research.google.com/github/MartinVIllesca/Ramos-Redes/blob/master/taller%20de%20proyecto/stanford-dogs-keras-vgg16(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# !tar -xvf '/content/drive/My Drive/data_EL5003/images.tar'

In [3]:
#-------Import Dependencies-------#
%matplotlib inline
import pandas as pd
import os,shutil,math,scipy,cv2
import numpy as np
import matplotlib.pyplot as plt
import random as rn


from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,roc_curve,auc

from PIL import Image
from PIL import Image as pil_image
from PIL import ImageDraw

from time import time
from glob import glob
from tqdm import tqdm
from skimage.io import imread
from IPython.display import SVG

from scipy import misc,ndimage
from scipy.ndimage.interpolation import zoom
# hay problemas con esta libreria
# from scipy.ndimage import imread

import keras
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras import layers
from keras.preprocessing.image import save_img
from keras.utils.vis_utils import model_to_dot
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.xception import Xception
from keras.applications.nasnet import NASNetMobile
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

Using TensorFlow backend.


In [0]:
from os import listdir, walk, path

In [0]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [0]:
def label_assignment(img,label):
    return label

def training_data(label,data_dir):
  x = []
  y = []
  for img in tqdm(os.listdir(data_dir)):
      label = label_assignment(img,label)
      path = os.path.join(data_dir,img)
      img = cv2.imread(path,cv2.IMREAD_COLOR)
      img = cv2.resize(img,(imgsize,imgsize))
      img = (np.array(img) / 255).astype('float32')
      x.append(np.array(img))
      y.append(str(label))

  return np.array(x), y

In [0]:
def read_data(n):
  F = []
  Y = []

  for idx, f in enumerate(listdir('./Images')):
    if idx == n: break
    y_ = f[10:]
    print('\r',y_)
    F.append(f)
    Y.append(y_)

  return Y, F

In [0]:
X = []
Z = []
Y = []
imgsize = 150
n_classes = 100

In [0]:
labels, files = read_data(n_classes)
n_classes = len(files)
for n in range(n_classes):
  x, y_ = training_data(labels[n], path.join('./Images/', files[n]))
  y = np.ones((len(y_),))*n
  if n == 0:
    X = x
    Z = y_
    Y = y
  else:
    X = np.concatenate([X, x])
    Z = np.concatenate([Z, y_])
    Y = np.concatenate([Y, y])
X = np.array(X)
print(X.shape, Y.shape, Z.shape)

 17%|█▋        | 34/206 [00:00<00:00, 338.26it/s]

 Tibetan_terrier
 Italian_greyhound
 Norfolk_terrier
 Kerry_blue_terrier
 Doberman
 Siberian_husky
 Sealyham_terrier
 West_Highland_white_terrier
 dhole
 Weimaraner
 schipperke
 Appenzeller
 malinois
 redbone
 Irish_water_spaniel
 bull_mastiff
 Labrador_retriever
 Mexican_hairless
 golden_retriever
 whippet
 collie
 Border_collie
 Saint_Bernard
 miniature_schnauzer
 Bouvier_des_Flandres
 Border_terrier
 Pekinese
 Dandie_Dinmont
 Chesapeake_Bay_retriever
 wire-haired_fox_terrier
 beagle
 Lakeland_terrier
 kuvasz
 German_shepherd
 Samoyed
 groenendael
 Australian_terrier
 pug
 Ibizan_hound
 miniature_poodle
 affenpinscher
 Newfoundland
 Gordon_setter
 malamute
 Bedlington_terrier
 English_foxhound
 Sussex_spaniel
 curly-coated_retriever
 EntleBucher
 American_Staffordshire_terrier
 cocker_spaniel
 cairn
 silky_terrier
 Cardigan
 Rhodesian_ridgeback
 basset
 Welsh_springer_spaniel
 Irish_wolfhound
 Boston_bull
 black-and-tan_coonhound
 dingo
 Great_Dane
 Afghan_hound
 Maltese_dog
 basenji

100%|██████████| 188/188 [00:00<00:00, 331.38it/s]

In [0]:
# label_encoder= LabelEncoder()
# Y = label_encoder.fit_transform(Z)
# Y = to_categorical(Y,n_classes)
Y = Y.astype(int)
# X = np.array(X)
# X=X/255


In [0]:
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(X) - 1)
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Dog: '+Z[l] + ' ' + str(Y[l]))
        
plt.tight_layout()

In [0]:
base_model = VGG16(include_top=False,
                  input_shape = (150,150,3),
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
for layer in base_model.layers:
    print(layer,layer.trainable)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
# model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(n_classes,activation='softmax'))
model.summary()

SVG(model_to_dot(model).create(prog='dot', format='svg'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, expand_nested=True)

In [0]:
#-----------Optimizers-----------#
opt = SGD(lr=1e-4,momentum=0.99)
opt1 = Adam(lr=1e-2)
#----------Compile---------------#
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

In [0]:
model.fit(X, Y,
          batch_size=32,
          epochs=50,
          validation_split=0.2)

In [0]:
# show_final_history(history)
# model.load_weights('/content/drive/My Drive/data_EL5003/model/model.h5')
# model_score = model.evaluate(X,Y)
# print("Model Test Loss:",model_score[0])
# print("Model Test Accuracy:",model_score[1])
# model.predict(X)